<a href="https://colab.research.google.com/github/Divyansh-Jain312/next_word_prediction-assignment-1/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Save the Model and Tokenizer


In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the text to be extracted
url = 'https://en.wikisource.org/wiki/The_Verdict'

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text from the page
text = soup.get_text()

# Save the text to a file
with open('Data.txt', 'w', encoding='utf-8') as file:
    file.write(text)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data
with open('Data.txt', 'r', encoding='utf-8') as file:
    data = file.read()

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

# Create input sequences using the tokens
input_sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=20, verbose=1)


Epoch 1/20
113/113 [==============================] - 15s 76ms/step - loss: 6.4545 - accuracy: 0.0407
Epoch 2/20
113/113 [==============================] - 9s 84ms/step - loss: 6.0435 - accuracy: 0.0485
Epoch 3/20
113/113 [==============================] - 9s 83ms/step - loss: 5.9395 - accuracy: 0.0485
Epoch 4/20
113/113 [==============================] - 8s 71ms/step - loss: 5.8064 - accuracy: 0.0482
Epoch 5/20
113/113 [==============================] - 10s 85ms/step - loss: 5.7179 - accuracy: 0.0530
Epoch 6/20
113/113 [==============================] - 10s 85ms/step - loss: 5.6505 - accuracy: 0.0538
Epoch 7/20
113/113 [==============================] - 8s 71ms/step - loss: 5.5882 - accuracy: 0.0588
Epoch 8/20
113/113 [==============================] - 9s 84ms/step - loss: 5.5277 - accuracy: 0.0616
Epoch 9/20
113/113 [==============================] - 10s 84ms/step - loss: 5.4576 - accuracy: 0.0630
Epoch 10/20
113/113 [==============================] - 8s 71ms/step - loss: 5.3841 - ac

In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
model.save('lstm_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pickle

with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
import os
os.listdir()

['.config', 'Data.txt', 'tokenizer.pkl', 'lstm_model.h5', 'sample_data']

In [ ]:
from google.colab import files

# Download the model file
files.download('lstm_model.h5')

# Download the tokenizer file
files.download('tokenizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Create the Streamlit App


In [4]:
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the trained model
model = tf.keras.models.load_model('lstm_model.h5')

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define the max sequence length
max_sequence_len = model.input_shape[1]  # This should be the same value as used during training

# Function to predict the next words
def predict_next_words(model, tokenizer, text, num_words):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == np.argmax(predicted):
                predicted_word = word
                break
        text += " " + predicted_word
    return text

# Streamlit app
st.title("Next Word Prediction")
input_text = st.text_input("Enter the initial text:")
num_words = st.number_input("Enter the number of words to predict:", min_value=1, max_value=100, value=1)

if st.button("Predict"):
    if input_text:
        predicted_text = predict_next_words(model, tokenizer, input_text, num_words)
        st.write(predicted_text)
    else:
        st.write("Please enter some text to start prediction.")







2024-06-26 17:55:56.383 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-26 17:55:56.386 Session state does not function when running a script without `streamlit run`


In [1]:
!pip install streamlit



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
!streamlit run app.py
